In [53]:
import wrf_management.utilities as ut
import importlib
importlib.reload(ut);
import wrf_management.project_global_constants as gc
importlib.reload(gc)
import wrf_management.geogrid as geo
import wrf_management.ungrib as un
importlib.reload(un)
import os
import sqlite3 as sq
import pandas as pd
import wrf_management.base_namelists.base_namelists as bn
importlib.reload(bn);
import f90nml

In [54]:
print(gc.RUN_NAME)
job = 'ungrib_surf'
file_types = ['surf_0','surf_1']
job = 'ungrib_press'
file_types=['press']
real = False

LIST_S_LINKS = [
    'ungrib.exe',
    'link_grib.csh',
    'ungrib'
]

LIST_H_LINKS = [
    'Vtable',
    'env_WRFv4.bash'
]


run_2019_02_20


In [55]:
con = sq.connect(gc.PATH_DB)
gc.PATH_DB

'/homeappl/home/aliagadi/saltena_2018/wrf_management/wrf_management/db_folder/wrf_man.sqlite'

In [56]:
importlib.reload(un)
run_row = un.get_run_row()
print(run_row)

job_row = un.get_next_row(job=job)
print(job_row)

un.update_run_table(val=job_row[job]+1,
                    job=job,
                    date=job_row['date']
                   )

job_path = un.getmk_job_path(run_row,job_row,job)
print(job_path)

untar_path = os.path.join(job_path,'untar')
print(untar_path)

conf_path = un.get_conf_path(run_row)
print(conf_path)

type_rows = pd.DataFrame([un.get_type_row(ft,job_row) for ft in file_types])
print(type_rows)

name_list = un.skim_namelist_copy(
    conf_path,job_path,date =job_row.date,prefix=job
)
print(name_list)

index                             1
run_name             run_2019_02_20
metgrid_path                geogrid
config_path          run_2019_02_20
data_path       runs/run_2019_02_20
Name: 0, dtype: object
date            2017-12-05 00:00:00
ungrib_surf                       0
ungrib_press                      0
ungrib_lake                       0
Name: 0, dtype: object
/wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20/2017_12_05/ungrib_press
/wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20/2017_12_05/ungrib_press/untar
/homeappl/home/aliagadi/saltena_2018/wrf_management/wrf_management/config_dir/run_2019_02_20
                  date  i  downloaded                      name  untarred  \
0  2017-12-05 00:00:00  4           1  cdas1.20171205.pgrbh.tar         0   

    type  
0  press  
OrderedDict([('share', Namelist([('wrf_core', 'ARW'), ('max_dom', 4), ('start_date', ['2017-12-05_00:00:00', '2017-12-05_00:00:00', '2017-12-05_00:00:00', '2017-12-05_00:00:0

In [57]:
if gc.ID=='taito_login':
    un.copy_hard_links(conf_path,job_path,LIST_H_LINKS)
    un.copy_soft_links(gc.PATH_WPS,job_path,LIST_S_LINKS)
    importlib.reload(un)
    un.untar_the_files(type_rows,job_path)

Vtable
env_WRFv4.bash
ungrib.exe
unilinking
link_grib.csh
ungrib
/wrk/aliagadi/DONOTREMOVE/wrf_management_data/press_tar/cdas1.20171205.pgrbh.tar


In [58]:
run_script = \
"""#!/bin/bash
cd {job_path}
./link_grib.csh ./untar/*
source ./env_WRFv4.bash 
./ungrib.exe
exit $?
""".format(job_path=job_path)
print(run_script)
bs_path = os.path.join(job_path,'run_me.sh')
bs_file = open(bs_path,'w')
bs_file.write(run_script)
bs_file.close()

#!/bin/bash
cd /wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20/2017_12_05/ungrib_press
./link_grib.csh ./untar/*
source ./env_WRFv4.bash 
./ungrib.exe
exit $?



In [ ]:
import subprocess as su
res = su.run(['/bin/bash',bs_path],stdout=su.PIPE,stderr=su.PIPE)

In [48]:
res.returncode

0

In [49]:
res.stdout

b' *** Starting program ungrib.exe ***\nStart_date =  2017-12-05_00:00:00 ,      End_date = 2017-12-05_18:00:00\noutput format is WPS\nPath to intermediate files is ./\nERROR: Grib2 file or date problem, stopping in edition_num.\n0\n'

In [50]:
un.update_run_table(val=100,
                    job=job,
                    date=job_row['date']
                   )